In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [54]:
import os
import pandas as pd

import numpy as np
import jax
jax.config.update('jax_platform_name', 'gpu')



In [55]:
fn_EcoCyc = '../data/sRNA/EcoCyc/EcoCyc_regulatory_RNAs.csv'
fn_RNAInter = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
fn_srnatarbase = os.path.join('..', 'data', 'sRNA', 'sRNATarBase', 'sRNATarBase.csv')
ecocyc = pd.read_csv(fn_EcoCyc)
rnainter = pd.read_csv(fn_RNAInter)
srnatarbase = pd.read_csv(fn_srnatarbase)
# data = pd.concat([pd.read_csv(fn_RNAInter), pd.read_csv(fn_EcoCyc)])
try:
    rnainter = rnainter.drop(columns=['Unnamed: 0'])
    srnatarbase = srnatarbase.drop(columns=['Unnamed: 0'])
except:
    pass


### Start with RNAInter

In [56]:
np.sum(rnainter['Sequence1'].isna()) + np.sum(rnainter['Sequence2'].isna())

2

In [57]:
data = pd.DataFrame()
rnainter['Database'] = 'RNAInter'
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor1.Symbol', 'Raw_ID1', 'Category1', 'Interactor1.Symbol', 'Sequence1', 'Database']):
    data[n] = rnainter.drop_duplicates(subset=['Interactor1.Symbol'])[o]
data2 = pd.DataFrame()
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2', 'Database']):
    data2[n] = rnainter.drop_duplicates(subset=['Interactor2.Symbol'])[o]

# data = pd.concat([data, rnainter.drop_duplicates(subset=['Interactor2.Symbol']).reset_index()[['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Sequence2']].rename(
#     columns={j: i for i, j in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2'])})], axis=0)
data = pd.concat([data, data2])

data = data[~data['Name'].isna()]
data = data.drop_duplicates(subset=['Name'])

In [58]:
data['Symbol'].value_counts()


Symbol
acnA    1
phoP    1
sibC    1
sibB    1
sibA    1
       ..
luxS    1
malK    1
manX    1
manY    1
symR    1
Name: count, Length: 276, dtype: int64

### Now EcoCyc

In [59]:
ecocyc.head()

,Unnamed: 0,Ontology - all instances of class from My Favorites,Matches,Common-Name,GO-Terms,Names,Object ID,Gene,Sequence - DNA sequence,Copy-Number,Common-Name.1
0,0,6S-RNA,Regulatory-RNAs,6S RNA,NaN,6S RNA // SsrS // Ssr,6S-RNA,EG30099,ATTTCTCTGAGATGTTCGCAAGCGGGCCAGTCCCCTGAGCCGATAT...,NaN,ssrS
1,1,RDLD-RNA,Regulatory-RNAs,antisense regulatory RNA RdlD,GO:2000766 // GO:0030371,antisense regulatory RNA RdlD // RdlD // RdlD ...,RDLD-RNA,G0-9042,GTCTAGAGTCAAGATTAGCCCCCGTGGTGTTGTCAGGTGCATACCT...,NaN,rdlD
2,2,RNA0-441,Regulatory-RNAs,arfA-derived small regulatory RNA,GO:0006970 // GO:0003729,<i>arfA</i>-derived small regulatory RNA,RNA0-441,G0-17130,AAGAAAGTAAATCACTTTTTTACCACTGGTCTTCTGCTTTCAGGCG...,NaN,yrdG
3,3,RNA0-409,Regulatory-RNAs,e14 prophage; small regulatory RNA C0293,GO:0035194 // GO:0040033 // GO:0003729,e14 prophage; small regulatory RNA C0293 // C0293,RNA0-409,G0-8897,TACTCACCCGGGACTCGCCAGGGGACAGCCAACAGGCATTGGGTGC...,NaN,C0293
4,4,RNA0-162,Regulatory-RNAs,putative antisense regulatory RNA RdlB,GO:1902201,putative antisense regulatory RNA RdlB // RdlB,RNA0-162,G0-9606,GTCTGGTTTCAAGATTAGCCCCCGTTCTGTTGTCAGGTTTTACCTC...,NaN,rdlB


In [60]:
ecocyc['Category'] = 'sRNA'
ecocyc['Database'] = 'EcoCyc'
ecocyc = ecocyc.drop_duplicates(subset=['Common-Name'])

data = pd.concat([data, ecocyc.rename(columns={j: i for i, j in zip(['Name', 'ID', 'Symbol', 'Sequence'], ['Common-Name', 'Object ID', 'Common-Name.1', 'Sequence - DNA sequence'])})[
    ['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database']]], axis=0)

### Finally sRNATarBase

In [61]:
srnatarbase_merged = pd.DataFrame()
srnatarbase_merged['Name'] = np.concatenate(
    [srnatarbase['sRNA'].unique(), srnatarbase['Target'].unique()], axis=0)
srnatarbase_merged['ID'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA ID'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target ID'].iloc[0])
srnatarbase_merged['Category'] = srnatarbase_merged['Name'].apply(
    lambda x: 'sRNA' if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Type'].iloc[0])
srnatarbase_merged['Symbol'] = srnatarbase_merged['Name']
srnatarbase_merged['Sequence'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA Sequence'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Sequence'].iloc[0])
srnatarbase_merged['Database'] = 'sRNATarBase'
srnatarbase_merged

,Name,ID,Category,Symbol,Sequence,Database
0,dsrA,"chromosome:NC_000913.3, Gene ID:946470",sRNA,dsrA,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,sRNATarBase
1,micF,"chromosome:NC_000913.3, Gene ID:2847742",sRNA,micF,GCTATCATCATTAACTTTATTTATTACCGTCATTCATTTCTGAATG...,sRNATarBase
2,micC,"chromosome:NC_000913.3, Gene ID:2847713",sRNA,micC,GTTATATGCCTTTATTGTCACAGATTTTATTTTCTGTTGGGCCATT...,sRNATarBase
3,oxyS,"chromosome:NC_000913.3, Gene ID:2847701",sRNA,oxyS,GAAACGGAGCGGCACCTCTTTTAACCCTTGAAGTCACTGCCCGTTT...,sRNATarBase
4,istR,"chromosome:NC_000913.3, Gene ID:5061525",sRNA,istR,GCACTAAATACGTCAAAATTCGTGCCGAAATTGCGCGTTCTGCGCG...,sRNATarBase
...,...,...,...,...,...,...
322,msrA,"chromosome:NC_000913.3, Gene ID:948734",mRNA,msrA,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase
323,hdeA,"chromosome:NC_000913.3, Gene ID:948025",mRNA,hdeA,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase
324,hdeB,"chromosome:NC_000913.3, Gene ID:948026",mRNA,hdeB,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase
325,fliC,"chromosome:NC_002695.1, Gene ID:",mRNA,fliC,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBase


In [62]:
data = pd.concat([data, srnatarbase_merged], axis=0)
data = data.reset_index().drop(columns='index')

# Delete duplicates responsibly

In [63]:
doubles = data['Symbol'].value_counts()[data['Symbol'].value_counts() > 1].index.to_list()
len(doubles)

274

In [66]:
for d in doubles:
    
    if len(data[data['Symbol'] == d]['Sequence'].unique()) == 1:
        print('Removing duplicate of ', d, data[data['Symbol'] == d]['Name'].unique())
        data = data.drop(index=data[(data['Symbol'] == d) & (data['Database'] != 'EcoCyc')].index[0])
    else:
        print('Did not remove', d)
        # print(data[data['Symbol'] == d]['Sequence'].values)
    
data = data[~data['Sequence'].isna()]
data
# data[data['Symbol'] == d].index[0]

Did not remove sokC
Did not remove sibB
Did not remove gadY
Did not remove gcvB
Did not remove rybB
Did not remove istR
Did not remove rseX
Did not remove rdlD
Did not remove rdlC
Did not remove rdlB
Did not remove sibE
Did not remove sibD
Did not remove sibC
Did not remove mgrR
Did not remove dsrA
Did not remove sokB
Did not remove micC
Did not remove micF
Did not remove ohsC
Did not remove omrA
Did not remove omrB
Did not remove oxyS
Did not remove mcaS
Did not remove sgrS
Did not remove sdsR
Did not remove ryhB
Did not remove fnrS
Did not remove rprA
Did not remove dicF
Did not remove cyaR
Did not remove arcZ
Did not remove arrS
Did not remove chiX
Did not remove symR
Did not remove acrZ
Did not remove ynfF
Did not remove yigL
Did not remove phoP
Removing duplicate of  sstT ['sstT']
Removing duplicate of  ykgE ['ykgE']
Removing duplicate of  oppA ['oppA']
Removing duplicate of  hdeB ['hdeB']
Removing duplicate of  hdeA ['hdeA']
Removing duplicate of  yobA ['yobA']
Did not remove sib

,Name,ID,Category,Symbol,Sequence,Database
0,acnA,NCBI:946724,mRNA,acnA,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,RNAInter
1,acrZ,NCBI:945365,mRNA,acrZ,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,RNAInter
2,arcZ,NCBI:2847690,sRNA,arcZ,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,RNAInter
3,argR,NCBI:947861,mRNA,argR,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,RNAInter
4,arrS,NCBI:11115378,sRNA,arrS,CTCTCCCTCTCTTTCTCTCTTCTCCGCGGCGATACGATCCGAGATG...,RNAInter
...,...,...,...,...,...,...
673,msrA,"chromosome:NC_000913.3, Gene ID:948734",mRNA,msrA,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase
674,hdeA,"chromosome:NC_000913.3, Gene ID:948025",mRNA,hdeA,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase
675,hdeB,"chromosome:NC_000913.3, Gene ID:948026",mRNA,hdeB,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase
676,fliC,"chromosome:NC_002695.1, Gene ID:",mRNA,fliC,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBase


In [38]:
data.to_csv('../data/sRNA/merged_EcoCyc_RNAInter_sRNATarBase.csv')